<a href="https://colab.research.google.com/github/Viswanth13/TripAdvisor_RAG/blob/main/RAG_VT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG

In [1]:
!pip install streamlit kaggle sentence-transformers pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [2]:
from google.colab import files
files.upload()  # Select kaggle.json
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d andrewmvd/trip-advisor-hotel-reviews
!unzip trip-advisor-hotel-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/trip-advisor-hotel-reviews
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
  0% 0.00/5.14M [00:00<?, ?B/s]
100% 5.14M/5.14M [00:00<00:00, 55.6MB/s]
Archive:  trip-advisor-hotel-reviews.zip
  inflating: tripadvisor_hotel_reviews.csv  


In [4]:
import pandas as pd
df = pd.read_csv('tripadvisor_hotel_reviews.csv')

In [5]:
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [6]:
!pip install regex

In [7]:
import regex as re
def clean_text(text):
    if pd.isna(text):
        return ""
    # Convert to string and lower case
    text = str(text).lower().strip()
    # Remove any extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Optionally, remove punctuation (you can adjust the pattern as needed)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [8]:
# Preprocessing

df = df.dropna(subset=["Review"]).reset_index(drop=True)

df["clean_review_text"] = df["Review"].apply(clean_text)

In [9]:
print("\nPreprocessed Data:")
df.head()


Preprocessed Data:


,Review,Rating,clean_review_text
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,nice rooms not 4 experience hotel monaco seatt...
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",5,great stay great stay went seahawk game awesom...


In [10]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model.
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = model.get_sentence_embedding_dimension()
print("Embedding dimension:", embedding_dim)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384


In [11]:
# Convert the "clean_review_text" column into a list and generate embeddings.
texts = df['clean_review_text'].tolist()
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
print("Generated embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/641 [00:00<?, ?it/s]

Generated embeddings shape: (20491, 384)


In [12]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.0 MB/s eta 0:00:00


In [13]:
# Create a FAISS index using L2 (Euclidean) distance.
index = faiss.IndexFlatL2(embedding_dim)
# Add the embeddings to the index.
index.add(embeddings)
print("FAISS index built with", index.ntotal, "vectors.")

FAISS index built with 20491 vectors.


In [14]:
# Retrieval function

def retrieve(query, top_k=5):
    # Convert query into an embedding vector.
    query_embedding = model.encode([query], convert_to_numpy=True)
    # Search the index; distances and indices are returned.
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for idx, dist in zip(indices[0], distances[0]):
        # Retrieve the original review, its cleaned version, and rating.
        result = {
            "Review": df.iloc[idx]['Review'],
            "clean_review_text": df.iloc[idx]['clean_review_text'],
            "Rating": df.iloc[idx]['Rating'],
            "Distance": dist
        }
        results.append(result)
    return results

In [15]:
# Testing retrieval pipeline

example_query = "exceptional service and amazing experience"
print(f"\nTop {3} results for query: '{example_query}'\n")
results = retrieve(example_query, top_k=3)
for i, res in enumerate(results, start=1):
    print(f"Result {i}:")
    print("Review:", res["Review"])
    print("Clean Review:", res["clean_review_text"])
    print("Rating:", res["Rating"])
    print("Distance:", res["Distance"])
    print()


Top 3 results for query: 'exceptional service and amazing experience'

Result 1:
Review: totally awesome, better expected service exceptional freindly, felt like family small staff went way little extra s not expect, 50th birthday wife went breakfast day leonor staff b.day cake singing happy birthday, totally blown away want thank leonor staff going way make day visit special time, place not 5 star staff 5+star clean bed comfy, location great just half block beach, understand hotel ranked 1 just friendly staff service, believe far best bang buck hawaii period, chuck ann kamloops b.c, canada,  
Clean Review: totally awesome better expected service exceptional freindly felt like family small staff went way little extra s not expect 50th birthday wife went breakfast day leonor staff bday cake singing happy birthday totally blown away want thank leonor staff going way make day visit special time place not 5 star staff 5star clean bed comfy location great just half block beach understand h

In [16]:
%%writefile app.py
import streamlit as st
import os
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import regex as re


filename = "tripadvisor_hotel_reviews.csv"

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

if os.path.exists(filename):
    st.write("Loading dataset from file...")
    df = pd.read_csv(filename)
    # If the "clean_review_text" column does not exist, create it from "Review"
    if "clean_review_text" not in df.columns:
        df["clean_review_text"] = df["Review"].apply(clean_text)
else:
    st.write("Dataset not found. Generating sample data...")
    sample_data = {
        "Review": [
            "I absolutely loved my stay at the Grand Beach Resort! The service was exceptional.",
            "The hotel was okay, but the food left much to be desired.",
            "An amazing experience overall; great views and friendly staff."
        ],
        "Rating": [5, 3, 4]
    }
    df = pd.DataFrame(sample_data)
    df["clean_review_text"] = df["Review"].apply(clean_text)

st.write("Dataset preview:")
st.dataframe(df.head())


# 2. Load Sentence Transformer Model and Generate Embeddings

st.write("Loading Sentence Transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = model.get_sentence_embedding_dimension()
st.write(f"Model loaded. Embedding dimension: {embedding_dim}")

# Convert the cleaned text into embeddings
texts = df['clean_review_text'].tolist()
with st.spinner("Generating embeddings..."):
    embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
st.write("Embeddings generated.")


# 3. Build FAISS Index

st.write("Building FAISS index...")
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)
st.write(f"FAISS index built with {index.ntotal} vectors.")


# 4. Define the Retrieval Function

def retrieve(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        result = {
            "Review": df.iloc[idx]['Review'],
            "Rating": df.iloc[idx]['Rating'],
            "Distance": dist
        }
        results.append(result)
    return results


# 5. Streamlit User Interface

st.title("TripAdvisor Reviews Retrieval App")
st.markdown("### Enter a query to search for relevant TripAdvisor reviews:")

# Text input for the user query
query = st.text_input("Your Query", "")

# Optional: Dropdown menu for filtering responses by Rating
unique_ratings = sorted(df["Rating"].unique().tolist())
selected_rating = st.selectbox("Filter by Rating", options=[None] + unique_ratings,
                                 format_func=lambda x: "All" if x is None else x)

# Optional: Checkbox to customize the number of results (top_k)
use_custom_topk = st.checkbox("Customize number of results", value=False)
if use_custom_topk:
    top_k = st.number_input("Number of Results", min_value=1, max_value=20, value=5, step=1)
else:
    top_k = 5

# Search button to trigger retrieval
if st.button("Search"):
    if query:
        with st.spinner("Retrieving results..."):
            results = retrieve(query, top_k=top_k)
        # Apply rating filter if one is selected
        if selected_rating is not None:
            results = [r for r in results if r["Rating"] == selected_rating]
        st.markdown("### Retrieved Results:")
        if results:
            for i, res in enumerate(results, start=1):
                st.markdown(f"**Result {i}:**")
                st.write("Review:", res["Review"])
                st.write("Rating:", res["Rating"])
                st.write("Distance:", round(res["Distance"], 4))
                st.markdown("---")
        else:
            st.write("No results found matching your query and filter.")
    else:
        st.warning("Please enter a query to search.")

# Optional: Button to explore additional related content
if st.button("Explore Related Content"):
    st.markdown("### Related Travel Resources:")
    st.markdown("- [TripAdvisor Travel Tips](https://www.tripadvisor.com/)")
    st.markdown("- [Lonely Planet Destinations](https://www.lonelyplanet.com/)")
    st.markdown("- [Nomadic Matt's Travel Blog](https://www.nomadicmatt.com/)")


Writing app.py


In [18]:
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token("2sbM5O5ELNlkZnKhFKkIA560kSm_5ajHjuaspCDzzAbBdeHCD")

# Start ngrok tunnel on port 8501 (as an integer) with HTTP protocol
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py

Streamlit app is live at: NgrokTunnel: "https://e512-34-16-246-96.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.246.96:8501

2025-02-11 19:22:42.289260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739301762.315146   11534 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739301762.323278   11534 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Batches: 100% 641/641 [00:28<00:00, 22.51it/s]
2025-02-11 19:23:18.633 Examining the path of torch.classes raised:
Traceback (most recent call last):
  F